In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("training_data_traducido_limpio.csv")
df

,nombre de empresa,descripción del trabajo,posicion_title,Descripción_length,modelo_esponse
0,google,calificaciones minimas licenciatura o experien...,especialista en ventas,2727,responsabilidades basicas responsable de expan...
1,manzana,descripcion como asc sera muy influyente en el...,consultor de soluciones de apple,828,responsabilidades centrales como asc sera muy ...
2,netflix,es un momento sorprendente para unirse a netfl...,coordinador de licencias productos de consumo,3205,core responsibilities help drive business by s...
3,robert mitad,descripcion disenadores web que buscan expandi...,disenador web,2489,responsabilidades basicas diseno de sitios web...
4,rastreador,en trackfive teniamos grandes objetivos estaba...,desarrollador web,3167,responsabilidades centrales construir y diseno...
...,...,...,...,...,...
848,menards,descripcion del trabajo tiempo parcial ganar m...,pasantia de gestion,1122,responsabilidades basicas las responsabilidade...
849,parker,responsabilidad el programa de pasantias de pa...,pasantia de recursos humanos corporativo duran...,3840,responsabilidades basicas asistir en funciones...
850,guardar el proyecto,el proyecto borgen es una campana nacional inn...,pasantia de escritor periodista,897,responsabilidades basicas escriba un articulo ...
851,destinos de wyndham,pon al mundo en vacaciones en wyndham destinat...,servicio al cliente ventas entrantes remoto,4604,responsabilidades centrales responda las llama...


buscar funciones generales para cada tipo de trqbajo para que el modelo pueda hacer una comparacion. 

Limpieza a todo


In [3]:
import pandas as pd
import re
import unidecode

# Diccionario leetspeak
leet_dict = {
    "4": "a", "3": "e", "1": "i", "0": "o", "@": "a", "$": "s",
    "5": "s", "7": "t", "8": "b", "9": "g"
}

def normalizar_leet(texto):
    texto = texto.lower()
    for simbolo, letra in leet_dict.items():
        texto = texto.replace(simbolo, letra)
    return texto

def limpiar_texto(texto):
    if isinstance(texto, str):
        texto = unidecode.unidecode(texto)                  # quitar acentos
        texto = normalizar_leet(texto)                      # reemplazar leet
        texto = re.sub(r"[^a-zA-Z0-9\s]", "", texto)        # eliminar símbolos
        texto = re.sub(r"\s+", " ", texto).strip()          # limpiar espacios
        return texto
    return texto

# === LIMPIEZA GENERAL ===
df = pd.read_csv("training_data_traducido.csv")
df_limpio = df.applymap(limpiar_texto)
df_limpio.to_csv("training_data_traducido_limpio.csv", index=False)
print("Archivo limpio guardado")


/tmp/ipykernel_14899/1975407879.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df.applymap(limpiar_texto)


Archivo limpio guardado


Este codigo hace correcion ortografica por bloques

In [ ]:
import pandas as pd
from textblob import TextBlob
import time
import os

ARCHIVO_ENTRADA = "training_data_traducido_limpio.csv"
ARCHIVO_SALIDA = "training_data_traducido_limpio_CORREGIDO.csv"
ARCHIVO_TEMP = "training_Progreso_Corregido.csv"
TAMANO_BLOQUE = 500
PAUSA_SEGUNDOS = 10

def corregir_ortografia(texto):
    try:
        return str(TextBlob(texto).correct())
    except:
        return texto

def procesar_celda(celda):
    return corregir_ortografia(celda)

# === CORRECCIÓN POR BLOQUES ===
if os.path.exists(ARCHIVO_TEMP):
    print("Reanudando desde archivo temporal...")
    df_corregido = pd.read_csv(ARCHIVO_TEMP)
    filas_procesadas = len(df_corregido)
else:
    print("Iniciando desde cero...")
    df_corregido = pd.DataFrame()
    filas_procesadas = 0

df = pd.read_csv(ARCHIVO_ENTRADA)
if "Descripción_length" in df.columns:
    df = df.drop(columns=["Descripción_length"])

total_filas = len(df)

while filas_procesadas < total_filas:
    fin = min(filas_procesadas + TAMANO_BLOQUE, total_filas)
    print(f"Corrigiendo filas {filas_procesadas} a {fin}...")

    bloque = df.iloc[filas_procesadas:fin].copy()
    bloque = bloque.applymap(procesar_celda)
    df_corregido = pd.concat([df_corregido, bloque], ignore_index=True)

    df_corregido.to_csv(ARCHIVO_TEMP, index=False)
    print(f"Progreso guardado: {ARCHIVO_TEMP}")

    filas_procesadas = fin
    if filas_procesadas < total_filas:
        print(f"Pausa de {PAUSA_SEGUNDOS} segundos...")
        time.sleep(PAUSA_SEGUNDOS)

df_corregido.to_csv(ARCHIVO_SALIDA, index=False)
print(f"Archivo final corregido guardado como: {ARCHIVO_SALIDA}")
os.remove(ARCHIVO_TEMP)


Iniciando desde cero...
Corrigiendo filas 0 a 500...


/tmp/ipykernel_17302/3460808003.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bloque = bloque.applymap(procesar_celda)
